In [1]:
import numpy as np
import cv2 as cv
from scipy.cluster.vq import kmeans2
from scipy.stats import mode
import matplotlib.pyplot as plt

In [2]:
from histogram import findHistogram, drawHist
import math
debug = False
full_tif = R"C:\data\grid\Fused_pic2_no_overlap_compute-1.tif"
# full_tif = R"C:\data\dsd_10\dsd_10\Out\img_stitch_bleng_vignettcorrection.png"
image = cv.imread(full_tif, cv.IMREAD_GRAYSCALE)
image = (image/(255/np.amax(image))).astype(np.uint8)
(h, w) = image.shape[:2]
(cX, cY) = (w // 2, h // 2)
# M = cv.getRotationMatrix2D((cX, cY), 10, 1.0)
# image = cv.warpAffine(image, M, (w, h))

In [10]:

#know that the image is in grayscale
# image = readAndNorm(full_tif, 0, 255)

hist = findHistogram(image, 256)

if debug:

    hist_w = 512
    hist_h = 400
    hist_img = np.zeros((hist_h, hist_w, 3), dtype=np.uint8)
 
    hist_img = drawHist(hist_img, hist, (255, 0, 0))
    cv.imshow("img", hist_img)
    cv.waitKey(0)


# (h, w) = image.shape[:2]
# (cX, cY) = (w // 2, h // 2)
# M = cv.getRotationMatrix2D((cX, cY), 45, 1.0)
# image = cv.warpAffine(image, M, (w, h))

height, width = image.shape[:2]
new_width = 800  # Set the desired width
scaling_factor = new_width / width
resized_image = cv.resize(image, (new_width, int(height * scaling_factor)))

resized_image[resized_image > 200] = 255
resized_image[resized_image < 50] = 0

cv.imshow("img", resized_image)
cv.waitKey(0)

resized_image_canny = cv.Canny(resized_image, 100, 200, None, 3, L2gradient=True)
cv.imshow("img", resized_image_canny)
cv.waitKey(0)

lines = cv.HoughLines(resized_image_canny, 1, np.pi / 90, 150, None, 0, 0)
cdst = cv.cvtColor(resized_image_canny, cv.COLOR_GRAY2BGR)
if lines is not None:
    for i in range(0, len(lines)):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        a = math.cos(theta)
        b = math.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
        pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
        cv.line(cdst, pt1, pt2, (0,0,255), 3, cv.LINE_AA)
    
cv.imshow("Detected Lines (in red) - Standard Hough Line Transform", cdst)
cv.waitKey(0)
cv.destroyAllWindows()


mask = np.where(resized_image > 7*int(255/10), 254, 0).astype(np.uint8)
kernel = cv.getStructuringElement(cv.MORPH_DILATE, (21, 21))
# print(kernel)
# kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))

# dilation = cv.dilate(mask, kernel, iterations = 1)
kernel = cv.getStructuringElement(cv.MORPH_RECT, (21, 21))
closing = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)
closing = cv.morphologyEx(closing, cv.MORPH_CLOSE, kernel)
cv.imshow("closing", closing)
cv.waitKey(0)
img_c = cv.cvtColor(resized_image, cv.COLOR_GRAY2BGR)

# img_c[closing > 250] = (255, 255, 255)
# img_c[closing > 250] = (0, 0, 0)
img_c[closing > 200] = 0
img_c[np.logical_and(img_c > 100, img_c < 200)] = 125
cv.imshow("img_c", img_c)
cv.waitKey(0)



dst = cv.Canny(img_c, 100, 200, None, 3, L2gradient=True)
cv.imshow("Detected Lines (in red) - Standard Hough Line Transform", dst)
cv.waitKey(0)
cv.destroyAllWindows()

# Copy edges to the images that will display the results in BGR
cdst = cv.cvtColor(dst, cv.COLOR_GRAY2BGR)




lines = cv.HoughLines(dst, 1, np.pi / 90, 200, None, 0, 0)

if lines is not None:
    for i in range(0, len(lines)):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        a = math.cos(theta)
        b = math.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
        pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
        cv.line(cdst, pt1, pt2, (0,0,255), 3, cv.LINE_AA)
        
cv.imshow("Detected Lines (in red) - Standard Hough Line Transform", cdst)
cv.waitKey(0)
cv.destroyAllWindows()

In [14]:
class Rectangle:


    def intersection(self, L1, L2):
        D  = -L1[0]  + L2[0]
        Dx = L1[1]  - L2[1]
        Dy = -L1[0] * L2[1] + L1[1] * L2[0]
        if D != 0:
            x = Dx / D
            y = Dy / D
            return y, x
        else:
            return False
    

    def intersectionFromPair(self, l_1, l_2):
        if math.sin(l_1[1]) == 0:
            if math.sin(l_2[1] == 0):
                return False
            x = -l_1[0] if l_1[1] > 1 else l_1[0] 
            y = l_2[0]
            return y, x
        if math.sin(l_2[1]) == 0:
            x = -l_2[0] if l_2[1] > 1 else l_2[0] 
            y = l_1[0]
            return y, x

        func_1 = (-math.cos(l_1[1])/math.sin(l_1[1]), l_1[0]/math.sin(l_1[1])) #y = l_1[0]*x+l_1[1]
        func_2 = (-math.cos(l_2[1])/math.sin(l_2[1]), l_2[0]/math.sin(l_2[1])) #y = l_1[0]*x+l_1[1]
        return self.intersection(func_1, func_2)


    def __init__(self, paralell_1, paralell_2) -> None:
        self.p_1 = paralell_1
        self.p_2 = paralell_2
        p_1 = self.intersectionFromPair(paralell_1[0], paralell_2[0])
        p_2 = self.intersectionFromPair(paralell_1[0], paralell_2[1])
        p_3 = self.intersectionFromPair(paralell_1[1], paralell_2[0])
        p_4 = self.intersectionFromPair(paralell_1[1], paralell_2[1])

        # diff_y1 = np.abs(p_1[0] - p_2[0])
        # diff_y2 = np.abs(p_3[0] - p_4[0])
        # diff_x1 = np.abs(p_1[1] - p_2[1])
        # diff_x2 = np.abs(p_3[1] - p_4[1])

        vertices = np.array([p_2[::-1], p_1[::-1], p_3[::-1], p_4[::-1]])
        self.vertices = vertices
        self.center = np.mean(vertices, axis=0)

        # Calculate the area of the quadrilateral
        self.area = 0.5 * np.abs(np.dot(vertices[:, 0], np.roll(vertices[:, 1], 1)) - np.dot(vertices[:, 1], np.roll(vertices[:, 0], 1)))
        self.base_rotation = np.amin([paralell_1[0][1], paralell_2[0][1]])



    def __str__(self) -> str:
        return f"h: {self.height} w: {self.width} LL: {self.lowerLeft} r: {self.base_rotation}"
    
    # def draw(self, img, color = (0, 0, 255)):
    #     return cv.rectangle(img, (np.uint8(self.lowerLeft[0]), np.uint8(self.lowerLeft[1])), (np.uint8(self.upperRight[0]), np.uint8(self.upperRight[1])), color, 1)

    def draw(self, img, color = (0, 0, 255)):
    
        pts = self.vertices.astype(np.int32)
        pts = pts.reshape((-1,1,2))
        return cv.polylines(img,[pts],True,color, 3)
    

In [20]:
import numpy as np

def delete_within_resolution(arr, resolution):
    result = np.copy(arr)
    i = 0
    while i < len(result):
        # Find the indices within the resolution around the current value
        indices_to_delete = np.where(np.abs(result - result[i]) <= resolution)[0]

        # Delete the indices within the resolution (excluding the current index)
        result = np.delete(result, indices_to_delete[indices_to_delete != i])
        i+=1
    return result

# Example usage:
arr = np.array([1, 2, 5, 8, 10, 12, 15])
resolution = 2

result = delete_within_resolution(arr, resolution)
print(result)

[ 1  5  8 12 15]


In [12]:

def drawLines(lines):
    cdst = cv.cvtColor(dst, cv.COLOR_GRAY2BGR)
    if lines is not None:
        for i in range(0, len(lines)):
            if len(np.array(lines).shape) == 3:
                rho = lines[i][0][0]
                theta = lines[i][0][1]
            else:
                rho = lines[i][0]
                theta = lines[i][1]
            a = math.cos(theta)
            b = math.sin(theta)
            x0 = a * rho
            y0 = b * rho
            pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
            pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
            cv.line(cdst, pt1, pt2, (0,0,255), 3, cv.LINE_AA)
            
    cv.imshow("Detected Lines (in red) - Standard Hough Line Transform", cdst)
    cv.waitKey(0)
    cv.destroyAllWindows()

drawLines([[[dst.shape[1]/2, 0]], [[dst.shape[0]/2, np.pi/2]]])

In [15]:
def generatePairs(lines1, lines2):
    #Generate a line pair where every line in lines1 pairs with every line in lines2
    pairs = []
    for line in lines1:
        for perpendicular in lines2:
            pairs.append([line[0], perpendicular[0]])
    return pairs

def delete_rho_within_resolution(arr, resolution):
    #generated with help from chatgpt
    result = np.copy(arr)
    i = 0
    while i < len(result):
        # Find the indices within the resolution around the current value
        
        indices_to_delete = np.where(np.abs(result[:, 0, 0] - result[i, 0, 0]) <= resolution)[0]

        # Delete the indices within the resolution (excluding the current index)
        result = np.delete(result, indices_to_delete[1:], axis=0)
        i+=1
    return result

def getCenter(angle, shape):
    if angle < np.pi/4:
        return (shape[1]/np.cos(angle))/2
    if angle == np.pi/4:
        return int(np.sqrt(shape[1]**2 + shape[0]**2)/2)
    if angle > np.pi/4 and angle < np.pi/2:
        return (shape[0]/np.cos(angle))/2
    if angle == np.pi/2:
        return shape[0]/2
    else:
        return getCenter(np.pi - angle, shape)

def split(arr, centrum):
    # Takes an array of lienes in polar space and groups them based on where the rhos has the highest increment
    
    MIN_INCREMENT = 80

    rhos = arr[:, 0, 0]
    #takin into account angles that wrap around having negative rhos
    if np.any(arr == 0) and np.any(arr > np.pi/2):
        rhos = np.where(arr[:, 0, 1] > np.pi/2, np.abs(rhos), rhos)

    sort_ids = np.argsort(rhos)
    increment = np.abs(rhos[sort_ids][1:]- rhos[sort_ids][:-1])




    if np.amax(increment) < MIN_INCREMENT:
        ids = np.where(rhos[sort_ids] > centrum)[0][0]
    else:
        ids = np.where(increment > np.amax(increment)/2)[0] + 1
    
    regions = [[], []]
    if len(ids) > 2:
        for i in ids:
            if rhos[sort_ids][i] > centrum:
                regions[0] = arr[sort_ids][:i]
                regions[1] = arr[sort_ids][i:]
                return regions
        print("No large increment over centrum found")
    regions[0] = arr[sort_ids][:ids[0]]
    regions[1] = arr[sort_ids][ids[0]:]
    return regions


def getMaximumAngleCoundWPerpendicular(hist, pm_degs=2):
    hist_out = hist[0].copy()
    for i in range(1, pm_degs + 1):
        hist_out += np.roll(hist[0], i)
        hist_out += np.roll(hist[0], -i)
    
    
    lower_half = hist_out[hist[1][:-1] < np.pi/2]
    upper_half = hist_out[hist[1][:-1] >= np.pi/2]


    return np.argmax(lower_half + upper_half)

def findSmallestParalell(region1, region2):
    id_a = len(region1) - 1
    id_b = 0
    inc_a = True
    while np.abs(region1[id_a][0][1] - region2[id_b][0][1]) < np.pi/45:
        print(id_a, id_b)
        if id_a < 0 or id_b >= len(region2):
            print("Could not find perfectly paralell lines")
            id_a = len(region1) - 1
            id_b = 0    
            break
        if inc_a:
            id_a -= 1
            inc_a = False
        else:
            id_b += 1
            inc_a = True
    return id_a, id_b

def getNextSimilarAngle(angle, arr, rev=False):

    print(len(arr))
    if rev:
        i = -1
        while abs(i) != len(arr) + 1:
            print(arr[i][0][1])
            print(angle)
            if np.abs(arr[i][0][1] - angle) < np.pi/45:
                return arr[i]
            i -= 1 
        print("could not find paralell line")
        return arr[-1]
    else:
        i = 0
        for i in range(len(arr)):
            if np.abs(arr[i][0][1] - angle) < np.pi/45:
                return arr[i]
        print("could not find paralell")
        return arr[0]






def sortPairs(lines, pm_degs=2):
    max_deg = np.pi
    # returns two arrays: degs (resolution of degrees) and l which is the lines on the degrees. 
    # degs[0] = 0 so l[0] all have a degree of 0 +/- max_deg/(num_degs-1)
    num_bins = (np.amax(lines[:, 0, 1]) + np.pi/90)/(np.pi/90)

    hist = np.histogram(lines[:, 0, 1], np.linspace(0, np.pi, 91, endpoint=True))
    

    max_degress_half_pi = getMaximumAngleCoundWPerpendicular(hist, pm_degs)
   
    #if argmax under pm_degs
    if max_degress_half_pi < pm_degs:
        upper_bound_id = pm_degs + max_degress_half_pi + 1 # less than this 
        lower_bound_id = len(hist[1]) - pm_degs #more than this

        upper_bound_deg = hist[1][upper_bound_id]
        lower_bound_deg = hist[1][lower_bound_id]
        print(upper_bound_deg)
        print(lower_bound_deg)
        print(upper_bound_deg + np.pi/2)
        print(lower_bound_deg - np.pi/2)
        mask_degrees_base = np.logical_or(lines[:, 0, 1] < upper_bound_deg, lines[:, 0, 1] > lower_bound_deg)
        mask_degrees_perpendicular = np.logical_and(lines[:, 0, 1] < upper_bound_deg + np.pi/2, lines[:, 0, 1] > lower_bound_deg - np.pi/2)


    elif max_degress_half_pi > 45 - pm_degs:
        # if argmax over 
        upper_bound_id = pm_degs + np.argmax(max_degress_half_pi) + 1 # less than this 
        lower_bound_id = np.argmax(max_degress_half_pi) - pm_degs #more than this

        upper_bound_deg = hist[i][upper_bound_id]
        lower_bound_deg = hist[i][lower_bound_id]

        mask_degrees_base = np.logical_and(lines[:, 0, 1] > lower_bound_deg, lines[:, 0, 1] < upper_bound_deg)
        mask_degrees_perpendicular = np.logical_or(lines[:, 0, 1] > lower_bound_deg + np.pi/2,\
                                                    lines[:, 0, 1] < upper_bound_deg - np.pi/2)

    else:

        #if argmax over pm_degs
        upper_bound_id = pm_degs + np.argmax(max_degress_half_pi) + 1 # less than this 
        lower_bound_id = np.argmax(max_degress_half_pi) - pm_degs #more than this

        upper_bound_deg = hist[i][upper_bound_id]
        lower_bound_deg = hist[i][lower_bound_id]

        mask_degrees_base = np.logical_and(lines[:, 0, 1] > lower_bound_deg, lines[:, 0, 1] < upper_bound_deg)
        mask_degrees_perpendicular = np.logical_and(lines[:, 0, 1] > lower_bound_deg + np.pi/2, lines[:, 0, 1] < upper_bound_deg + np.pi/2)

    
    base = lines[mask_degrees_base]
    drawLines(base)
    # print(len(base))
    base = delete_rho_within_resolution(base, 10)
    # print(len(base))
    perpendicular = lines[mask_degrees_perpendicular]
    drawLines(perpendicular)
    # print(len(perpendicular))
    perpendicular = delete_rho_within_resolution(perpendicular, 10)
    
    prominent_angle, _ = mode(base)

    base_regions = split(base, prominent_angle[0])
    # for region in base_regions:
    #     drawLines(region)
    prominent_angle, _ = mode(perpendicular)
    perpendicular_regions = split(perpendicular, prominent_angle[0])
    # for region in perpendicular_regions:
    #     drawLines(region)

    if len(base_regions) == 1:
        print("could not form regions in base")
    if len(perpendicular_regions) == 1:
        print("could not form regions in perpendicular")

    found_rectangle = True
    #find smallest region with ish paralell lines:

    id_a, id_b = findSmallestParalell(base_regions[0], base_regions[1])
    if id_a < 0 or id_b >= len(base_regions[1]):
        print("Could not find two rectangles")
        found_rectangle = False
    print(id_a)
    p_1 = [base_regions[0][id_a][0], base_regions[1][id_b][0]]

    base_regions[0] = base_regions[0][:id_a]
    base_regions[1] = base_regions[1][id_b + 1:]

    id_a, id_b = findSmallestParalell(perpendicular_regions[0], perpendicular_regions[1])
    p_2 = [perpendicular_regions[0][id_a][0], perpendicular_regions[1][id_b][0]]
    perpendicular_regions[0] = perpendicular_regions[0][:id_a]
    perpendicular_regions[1] = perpendicular_regions[1][id_b + 1:]
    
    if id_a < 0 or id_b >= len(perpendicular_regions[1]):
        print("Could not find two rectangles")
        found_rectangle = False

    smallest_r = Rectangle(p_1, p_2)
    drawLines([p_1[0], p_1[1], p_2[0], p_2[1]])
    if found_rectangle == False:
        return smallest_r, None
    
    a = getNextSimilarAngle(p_1[0][1], base_regions[0], rev=True)
    b = getNextSimilarAngle(p_1[1][1], base_regions[1], rev=False)
    c = getNextSimilarAngle(p_2[0][1], perpendicular_regions[0], rev=True)
    d = getNextSimilarAngle(p_2[1][1], perpendicular_regions[1], rev=False)

    larger_r = Rectangle([a[0], b[0]], [c[0], d[0]])


    return smallest_r, larger_r

s_r, l_r = sortPairs(lines)

0.10471975511965978
3.1066860685499065
1.6755160819145563
1.53588974175501
1 0
0 0
0 1
-1 1
Could not find perfectly paralell lines
1
1 0
0 0
0 1
-1 1
Could not find perfectly paralell lines
1
0.0
0.0
1
1
1.6406095
1.5707964
3


In [17]:
img = cv.resize(image, (new_width, int(height * scaling_factor)))
cdst = cv.cvtColor(dst, cv.COLOR_GRAY2BGR)


cdst = s_r.draw(cdst, (0, 255, 0))
cdst = l_r.draw(cdst, (0, 0, 255))
    
            
cv.imshow("Detected Lines (in red) - Standard Hough Line Transform", cdst)
cv.waitKey(0)
cv.destroyAllWindows()

In [25]:
min_area_r = None
for r in rs:
    if np.all(np.abs(dst.shape - 2*r.center) < 200):
        if min_area_r is None:
            min_area_r = r
        elif min_area_r.area > r.area:
            min_area_r = r        
        vertices = r.vertices
        cdst = cv.cvtColor(dst, cv.COLOR_GRAY2BGR)
        cdst = r.draw(cdst)
        for line in np.concatenate((r.p_1, r.p_2)):
            rho = line[0]
            theta = line[1]
            a = math.cos(theta)
            b = math.sin(theta)
            x0 = a * rho
            y0 = b * rho
            pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
            pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
            cv.line(cdst, pt1, pt2, (0,0,255), 3, cv.LINE_AA)
        cv.imshow("img", cdst)
        cv.waitKey(0)
    # break

cv.destroyAllWindows()

In [67]:
def sortPairs(lines, num_degs=91, pm_degs=2):
    max_deg = np.pi
    # returns two arrays: degs (resolution of degrees) and l which is the lines on the degrees. 
    # degs[0] = 0 so l[0] all have a degree of 0 +/- max_deg/(num_degs-1)
    degs = np.linspace(0, max_deg, num_degs)
    l = [[] for _ in range(len(degs)-1)]
    print(np.histogram(lines[:, 0, 1], 90))
    #iterate through all the lines in
    for i in range(len(lines)):
        theta = lines[i][0][1]
       
        rho = lines[i][0][0]
        #Finds the index of theta in the array "degs"
        id = np.where(degs > theta)[0][0]
        if id == num_degs -1:
            id = id - 1
        if len(np.where(abs(l[id] - rho) < 20)[0]) == 0: # Filter out other lines with the same theta and an approximate rho
            l[id].append(lines[i][0])

    len_l = np.array([len(a) for a in l])
    len_w_pm = []
    pm_half = int(pm_degs/2)
    half_len = int(len(len_l)/2)
    for i in range(half_len):
        if i < pm_half:
            s = np.sum(len_l[0:i + pm_half])
            print(np.sum(len_l[-(pm_half - i):]), len_l[-1])
            s += np.sum(len_l[-(pm_half - i):])

            s += np.sum(len_l[half_len + i - pm_half:half_len + i + pm_half])

            # print(-(pm_half - i), i + pm_half)
            # print(half_len + i - pm_half, half_len + i + pm_half)
        elif i > half_len - pm_half:
            s = np.sum(len_l[i - pm_half: i + pm_half])
            s += np.sum(len_l[(i - half_len):])
            s += np.sum(len_l[0:pm_degs - (i - half_len)])

            # print(i - pm_half, i + pm_half)
            # print(half_len + i - pm_half, half_len + i + pm_half - len(len_l))

        else:
            s = np.sum(len_l[i - pm_half: i + pm_half])
            s += np.sum(len_l[half_len + i - pm_half:half_len + i + pm_half])
            
            # print(i - pm_half, i + pm_half)
            # print(half_len + i - pm_half, half_len + i + pm_half)

        len_w_pm.append(s)

    print(np.argmax(np.array(len_w_pm)))
    print(len_w_pm)
    max_degs = np.argmax(np.array(len_w_pm))
    
    if max_degs - pm_half < 0:    
        deg_u = degs[max_degs + pm_half] 
        deg_o = degs[max_degs - pm_half]
        mask = np.logical_or(lines[:, 0, 1] < deg_u, lines[:, 0, 1] > deg_o)
        l_out = lines[mask]

    elif max_degs + pm_half > len(len_w_pm):

        deg_u = degs[len(len_w_pm) - (max_degs + pm_half)]
        deg_o = degs[max_degs - pm_half]

        mask = np.logical_or(lines[:, 0, 1] < deg_u, lines[:, 0, 1] > deg_o)
        l_out = lines[mask]
    else:
        mask = np.logical_and(lines[:, 0, 1] < degs[max_degs + pm_half], lines[:, 0, 1] > degs[max_degs - pm_half])
        l_out = lines[mask]

    # print(l_out.reshape((l_out.shape[0], l_out.shape[2])))
    

    return l_out.reshape((l_out.shape[0], l_out.shape[2]))

n_lines = sortPairs(lines)

(array([33, 15, 17,  9,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  3,  9, 13, 21,  8,  9,  7,  2,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  2, 16, 19, 32], dtype=int64), array([0.        , 0.03451873, 0.06903747, 0.1035562 , 0.13807493,
       0.17259367, 0.2071124 , 0.24163114, 0.27614987, 0.31066862,
       0.34518734, 0.37970608, 0.4142248 , 0.44874355, 0.48326227,
       0.517781  , 0.55229974, 0.5868185 , 0.62133723, 0.65585595,
       0.6903747 , 0.72489345, 0.75941217, 0.7939309 , 0.8284496 ,
       0.8629684 , 0.8974871 , 0.9320058 , 0.96652454, 1.0010433 ,
       1.035562  , 1.0700808 , 1.1045995 , 1.1391182 , 1.173637  ,
       1.2081558 , 1.2426745 , 1.2771932 , 1.3117119 , 1.3462306 ,
       1.3807493 , 1.4152681 , 1.4497869 , 1.4

In [5]:
def getMax(degs, l):
    window = (-1, 0, 1)
    num = []
    vals = []
    for i in range(len(l)):

        #check all values in window
        
        local_length = sum([len(l[i + w]) if (i+w) < len(l) else len([i+w - len(l)]) for w in window])

        if local_length != 0:
            num.append(local_length)
            vals.append(degs[i+window[len(window)//2]])



    return num, vals
    


In [6]:
degs, l = sortPairs(lines)
num, vals = getMax(degs, l)
print(vals)
print(degs)
max_num = 0
max_deg = 0
for i, val in enumerate(vals):
    print(val + np.pi/2)
    try:
        id = vals.index(val + np.pi/2)
        if max_num < num[i] + num[id]:
            max_num = num[i] + num[id]
            max_deg = val
    except ValueError:
        continue



[0.0, 0.03490658503988659, 0.06981317007977318, 0.10471975511965978, 0.13962634015954636, 0.17453292519943295, 1.4660765716752369, 1.5009831567151235, 1.53588974175501, 1.5707963267948966, 1.6057029118347832, 1.6406094968746698, 1.6755160819145565, 1.710422666954443, 1.7453292519943295, 1.7802358370342162, 2.9670597283903604, 3.001966313430247, 3.036872898470133, 3.07177948351002, 3.1066860685499065]
[0.         0.03490659 0.06981317 0.10471976 0.13962634 0.17453293
 0.20943951 0.2443461  0.27925268 0.31415927 0.34906585 0.38397244
 0.41887902 0.45378561 0.48869219 0.52359878 0.55850536 0.59341195
 0.62831853 0.66322512 0.6981317  0.73303829 0.76794487 0.80285146
 0.83775804 0.87266463 0.90757121 0.9424778  0.97738438 1.01229097
 1.04719755 1.08210414 1.11701072 1.15191731 1.18682389 1.22173048
 1.25663706 1.29154365 1.32645023 1.36135682 1.3962634  1.43116999
 1.46607657 1.50098316 1.53588974 1.57079633 1.60570291 1.6406095
 1.67551608 1.71042267 1.74532925 1.78023584 1.81514242 1.850

In [140]:
class Rectangle:


    def intersection(self, L1, L2):
        D  = -L1[0]  + L2[0]
        Dx = L1[1]  - L2[1]
        Dy = -L1[0] * L2[1] + L1[1] * L2[0]
        if D != 0:
            x = Dx / D
            y = Dy / D
            return y, x
        else:
            return False
    

    def intersectionFromPair(self, l_1, l_2):
        if math.sin(l_1[1]) == 0:
            if math.sin(l_2[1] == 0):
                return False
            x = -l_1[0] if l_1[1] > 1 else l_1[0] 
            y = l_2[0]
            return y, x
        if math.sin(l_2[1]) == 0:
            x = -l_2[0] if l_2[1] > 1 else l_2[0] 
            y = l_1[0]
            return y, x

        func_1 = (-math.cos(l_1[1])/math.sin(l_1[1]), l_1[0]/math.sin(l_1[1])) #y = l_1[0]*x+l_1[1]
        func_2 = (-math.cos(l_2[1])/math.sin(l_2[1]), l_2[0]/math.sin(l_2[1])) #y = l_1[0]*x+l_1[1]
        return self.intersection(func_1, func_2)

    def _findBaseLine(self, pair_1, pair_2):
        lowest_line = None

        for line in [pair_1[0], pair_1[1], pair_2[0], pair_2[1]]:
            print(line[1])
            if line[1] > np.pi/6 and line[1] < 5*np.pi/6:
                if lowest_line is None:
                    lowest_line = line
                elif line[0] < lowest_line[0]:
                    lowest_line = line 
        return lowest_line



    def __init__(self, pair_1, pair_2) -> None:

        base_line = self._findBaseLine(pair_1, pair_2)
        print()
        print(base_line)
        if np.all(base_line == pair_1[0]) or np.all(base_line == pair_1[1]):
            upper_line = pair_1[0] if np.all(base_line == pair_1[1]) else pair_1[1]
            print(upper_line)
            print(pair_2)
            print()
            if pair_2[0][0] > pair_2[1][0]:
                lowerLeft = self.intersectionFromPair(base_line, pair_2[1])
                lowerRight = self.intersectionFromPair(base_line, pair_2[0])
                upperLeft = self.intersectionFromPair(upper_line, pair_2[1])
                upperRight = self.intersectionFromPair(upper_line, pair_2[0])
            else:
                lowerLeft = self.intersectionFromPair(base_line, pair_2[0])
                lowerRight = self.intersectionFromPair(base_line, pair_2[1])
                upperLeft = self.intersectionFromPair(upper_line, pair_2[0])
                upperRight = self.intersectionFromPair(upper_line, pair_2[1])
        else:
            upper_line = pair_2[0] if np.all(base_line == pair_2[1]) else pair_2[1]
            if pair_1[0][0] > pair_1[1][0]:
                lowerLeft = self.intersectionFromPair(base_line, pair_1[1])
                lowerRight = self.intersectionFromPair(base_line, pair_1[0])
                upperLeft = self.intersectionFromPair(upper_line, pair_1[1])
                upperRight = self.intersectionFromPair(upper_line, pair_1[0])
            else:
                lowerLeft = self.intersectionFromPair(base_line, pair_1[0])
                lowerRight = self.intersectionFromPair(base_line, pair_1[1])
                upperLeft = self.intersectionFromPair(upper_line, pair_1[0])
                upperRight = self.intersectionFromPair(upper_line, pair_1[1])

        base_rotation = base_line[1]
        
        height = int(np.sqrt((max(lowerLeft[0], upperLeft[0])-min(lowerLeft[0], upperLeft[0]))**2 + (max(lowerLeft[1], upperLeft[1])-min(lowerLeft[1], upperLeft[1]))**2))
        width = int(np.sqrt((max(lowerLeft[0], lowerRight[0])-min(lowerLeft[0], lowerRight[0]))**2 + (max(lowerLeft[1], lowerRight[1])-min(lowerLeft[1], lowerRight[1]))**2)) 
        area = int(height*width)

        centrum = np.array([(max(upperRight[0], upperLeft[0], lowerLeft[0], lowerRight[0]) - min(upperRight[0], upperLeft[0], lowerLeft[0], lowerRight[0]))/2,\
                   (max(upperRight[1], upperLeft[1], lowerLeft[1], lowerRight[1]) - min(upperRight[1], upperLeft[1], lowerLeft[1], lowerRight[1]))/2])

        
        self.lowerLeft = lowerLeft  
        self.lowerRight = lowerRight
        self.upperLeft = upperLeft  
        self.upperRight = upperRight
        self.area = area
        self.centrum = centrum
        self.height = height
        self.width = width
        self.base_rotation = base_rotation




    def __str__(self) -> str:
        return f"h: {self.height} w: {self.width} LL: {self.lowerLeft} r: {self.base_rotation}"
    
    # def draw(self, img, color = (0, 0, 255)):
    #     return cv.rectangle(img, (np.uint8(self.lowerLeft[0]), np.uint8(self.lowerLeft[1])), (np.uint8(self.upperRight[0]), np.uint8(self.upperRight[1])), color, 1)

    def isOutside(self, shape):
        print(self.lowerLeft)
        print(self.lowerRight)
        print(self.upperRight)
        print(self.upperLeft)

        ll = self.lowerLeft[::-1]
        lr = self.lowerRight[::-1]
        ur = self.upperRight[::-1]
        ul = self.upperLeft[::-1]
        if ul[0] < 0 or ul[1] < 0:
            return True
        if ur[0] < 0 or ur[1] > shape[1]:
            return True
        if lr[0] > shape[0] or lr[1] > shape[1]:
            return True
        if ll[0] < 0 or ll[1] > shape[1]:
            return True
        return False



    def draw(self, img, color = (0, 0, 255)):
        print("lower left ", self.lowerLeft)
        print("lower right ", self.lowerRight)
        print("upper right ", self.upperRight)
        print("upper left ", self.upperLeft)



        pts = np.array([self.upperLeft[::-1], self.lowerLeft[::-1], self.lowerRight[::-1], self.upperRight[::-1]], np.int32)
        pts = pts.reshape((-1,1,2))
        return cv.polylines(img,[pts],True,color, 3)
    

In [51]:
print(lines[:, 0, 0])

[  50.   48.  780.  783.  777.  -14.  738.  763.  785.  791.  157.  773.
 -759.  768.  789.   11.  794.  161. -762.  -16.  766.  788.  910.  796.
  800.  790.   37.   45.   40.   35.   42.  689. -765.   -2.  115. -756.
  771.    9.  -20.   -9.    5.   56.   14.  803.  740.  -12.    2.   -6.
  906.   32.   58.   54.   30.    3.   16.  -22.    0. -754. -768.   20.
  -18.   23. -751.    1.   30.  159.   27.   16.   51.    5.  992.  880.
   62.   14.   68.  111.   63. -773. -771.   35.    7.  -24.   40.   18.
   66.   10.   13.   21.   11.   18.  113.    6.    2.   71.   20.   31.
  810.   29.  691.   38.   15.    5.   27.   16.   64.   33.   43.   25.
   23.   27. -740. -746.   45.   28.   30.    0. -748.  -28.   75.   26.
    3.  -30.  819. -747. -734.  817.   12.   -7.   42. -730.   23.   56.
   46. -744. -737.  -10.   -3.  815.  812.   25.   47. -732. -727.  -32.
   27. -779.   84.  822.   99.   50.  -34.   52.   53.   29.   78.   97.
  893.   26.   41.   54.  -13.   59.  827.   37.   

In [54]:
def findParalellFromSep(degs, rho, resolution=np.pi/90, rho_resolution=40):

    pairs = []
    for i in range(len(degs)):
        if len(rho[i]) <= 1:
            continue
        k = 0
        while k < len(rho[i]) - 1:
            line1 = rho[i][k]
            j = k + 1
            while j < len(rho[i]):
                line2 = rho[i][j]
                if abs(line1[0] - line2[0]) < rho_resolution:
                    rho[i].pop(j)
                    continue

                pairs.append([line1, line2])
                j += 1
            k += 1
    print(pairs)
    return pairs


# pairs = findParalellFromSep(lines[:, 0, 1], lines[:, 0, 0])


def findParalell(lines, resolution=np.pi/90, rho_resolution=40):
    """Finds all the paralell lines where paralell means that deg_1 = deg_2 +/- resolution"""
    pairs = []
    lines = list(lines)
    i = 0
    while i < len(lines) - 1: 
        deg_1 = lines[i][0][1]

        j = i + 1
        while j < len(lines):
            deg_cmp = abs(deg_1 - lines[j][0][1])

            if deg_cmp > resolution: # check if the lines have approximate equal degree if not continue
                j += 1
                continue
            if abs(lines[i][0][0] - lines[j][0][0]) < rho_resolution: #See if the rhos are not approximately equal
                lines.pop(j)
                continue

            pairs.append([lines[i][0], lines[j][0]])
            j += 1
        
        i += 1
    return pairs

pairs = findParalell(lines)

def findRectFromPairs(pairs, resolution=np.pi/90):
    rectangles = []
    for i in range(len(pairs)):

        for j in range(i+1, len(pairs)):
            if abs(pairs[i][0][1] - pairs[j][0][1]) < resolution or \
               abs(pairs[i][0][1] - pairs[j][1][1]) < resolution or \
               abs(pairs[i][1][1] - pairs[j][0][1]) < resolution or \
               abs(pairs[i][1][1] - pairs[j][1][1]) < resolution:
                continue
            if (pairs[i][0][1] < np.pi/6 or pairs[i][0][1] > 5*np.pi/6) and \
               (pairs[i][1][1] < np.pi/6 or pairs[i][1][1] > 5*np.pi/6) and \
               (pairs[j][0][1] < np.pi/6 or pairs[j][0][1] > 5*np.pi/6) and \
               (pairs[j][1][1] < np.pi/6 or pairs[j][1][1] > 5*np.pi/6):
                continue
            r = Rectangle(pairs[i], pairs[j])
            rectangles.append(r)

    return rectangles


    

In [122]:
cdst = cv.cvtColor(dst, cv.COLOR_GRAY2BGR)
lines = [[[-700, 3.10]], [[700, 0]]]
if lines is not None:
    for i in range(0, len(lines)):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        a = math.cos(theta)
        b = math.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
        pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
        cv.line(cdst, pt1, pt2, (0,0,255), 3, cv.LINE_AA)
        
cv.imshow("Detected Lines (in red) - Standard Hough Line Transform", cdst)
cv.waitKey(0)
cv.destroyAllWindows()

In [105]:



# pairs = findParalell(lines)

print(len(pairs))
rectangles = findRectFromPairs(pairs)



108
0.0
1.5707964
3.106686
1.6406095

[30.         1.6406095]
0.0
1.5707964
3.106686
1.6406095

[738.          1.5707964]
[50.  0.]
[array([-14.      ,   3.106686], dtype=float32), array([880.       ,   1.6406095], dtype=float32)]

0.0
1.5707964
3.106686
1.5358897

[53.         1.5358897]
0.0
1.5707964
3.106686
1.6406095

[-16.          1.6406095]
0.0
1.5707964
3.106686
1.6406095

[30.         1.6406095]
0.0
1.5707964
3.106686
1.6406095

[738.          1.5707964]
[50.  0.]
[array([-759.      ,    3.106686], dtype=float32), array([880.       ,   1.6406095], dtype=float32)]

0.0
1.5707964
3.106686
1.5358897

[53.         1.5358897]
0.0
1.5707964
3.106686
1.6406095

[-16.          1.6406095]
0.0
1.5707964
0.06981317
1.6406095

[30.         1.6406095]
0.0
1.5707964
0.06981317
1.6406095

[738.          1.5707964]
[50.  0.]
[array([7.100000e+01, 6.981317e-02], dtype=float32), array([880.       ,   1.6406095], dtype=float32)]

0.0
1.5707964
0.06981317
1.5358897

[53.         1.5358897]
0.0
1.

In [106]:
# pairs = findParalell(lines)

def perpendicular(alpha1, alpha2, rad_threshold):
    if np.abs(np.abs(alpha1 - alpha2) - np.pi/2) < rad_threshold:
        return True
    return False

def addline(line, cdst):
    rho = line[0]
    theta = line[1]
    a = math.cos(theta)
    b = math.sin(theta)
    x0 = a * rho
    y0 = b * rho
    pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
    pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
    cv.line(cdst, pt1, pt2, (0,0,255), 3, cv.LINE_AA)
    return cdst
    
cdst = cv.cvtColor(dst, cv.COLOR_GRAY2BGR)

print(pairs)
rec_out = []
max_rec = None
min_rec = None
for j in range(len(pairs)):
    print(j)
    alpha1 = (pairs[j][0][1] + pairs[j][1][1])/2
    zetha1 = (pairs[j][0][0] + pairs[j][1][0])/2 
    for i in range(j+1, len(pairs)):
        print(i)
        alpha2 = (pairs[i][0][1] + pairs[i][1][1])/2
        zetha2 = (pairs[i][0][0] + pairs[i][1][0])/2 
        if perpendicular(alpha1, alpha2, 5*np.pi/180): # if difference in degrees of pairs is less then 5 deg they are choosen as a rectangle
            r = Rectangle(pairs[i], pairs[j])
            # if r.isOutside(cdst.shape):
            #     continue
            if max_rec is None:
                max_rec = r
            elif max_rec.area < r.area:
                max_rec = r
            elif min_rec is None:
                min_rec = r
            elif min_rec.area > r.area:
                min_rec = r
            
            rec_out.append(r)


            # img_temp = cdst.copy()
            # addline(pairs[j][0], img_temp)
            # addline(pairs[j][1], img_temp)
            # addline(pairs[i][0], img_temp)
            # addline(pairs[i][1], img_temp)
            # img_temp = r.draw(img_temp)
            # cv.imshow("img", img_temp)
            # cv.waitKey(0)
            # break
# cv.destroyAllWindows()
min_c_rec = None
for r in rec_out:
    if r == max_rec:
        continue
    if r.area >= max_rec.area:
        continue
    # if np.any(np.abs(r.centrum - max_rec.centrum) > 100):
    if np.abs(r.centrum[1] - max_rec.centrum[1]) < 50:
        continue
    if np.abs(r.centrum[0] - max_rec.centrum[0]) > max_rec.height/2:
        continue

    if min_c_rec == None:
        min_c_rec = r
    elif min_c_rec.area > r.area:
        min_c_rec = r

    # img_temp = cdst.copy()
    # img_temp = r.draw(img_temp)
    # cv.imshow("img", img_temp)
    # cv.waitKey(0)

img_temp = cdst.copy()
img_temp = max_rec.draw(img_temp, (0, 0, 255))
img_temp = min_rec.draw(img_temp, (0, 255, 0))
img_temp = min_c_rec.draw(img_temp, (255, 0, 0))
print(max_rec.centrum)
print(min_c_rec.centrum)
# print(max_rec.area)
# print(max_rec.isOutside(img_temp.shape))
cv.imshow("img", img_temp)
cv.waitKey(0)
cv.destroyAllWindows()




[[array([50.,  0.], dtype=float32), array([738.       ,   1.5707964], dtype=float32)], [array([50.,  0.], dtype=float32), array([157.       ,   1.5707964], dtype=float32)], [array([50.,  0.], dtype=float32), array([910.       ,   1.5707964], dtype=float32)], [array([50.,  0.], dtype=float32), array([30.       ,  1.6406095], dtype=float32)], [array([50.,  0.], dtype=float32), array([992.       ,   1.5707964], dtype=float32)], [array([50.,  0.], dtype=float32), array([880.       ,   1.6406095], dtype=float32)], [array([50.,  0.], dtype=float32), array([6.       , 1.5707964], dtype=float32)], [array([50.,  0.], dtype=float32), array([53.       ,  1.5358897], dtype=float32)], [array([50.,  0.], dtype=float32), array([-16.       ,   1.6406095], dtype=float32)], [array([780.,   0.], dtype=float32), array([738.       ,   1.5707964], dtype=float32)], [array([780.,   0.], dtype=float32), array([157.       ,   1.5707964], dtype=float32)], [array([780.,   0.], dtype=float32), array([910.       , 

TypeError: 'bool' object is not subscriptable

In [107]:
i = 0

for r in rectangles:
    img = cv.cvtColor(dst, cv.COLOR_GRAY2BGR)
    img = r.draw(img)
    cv.imshow("img", img)
    cv.waitKey(0)
    i +=1
    if i == 50:
        break

cv.destroyAllWindows()

lower left  (30.0, 50.0)
lower right  (738.0004425266627, 10123.829553977943)
upper right  (738.0000017388409, 39.78002702526123)
upper left  (-14.0, 50.0)
lower left  (738.0000017388409, 39.78002702526123)
lower right  (737.9999098922048, -2061.4260041225234)
upper right  (880.0, 50.0)
upper left  (-14.0, 50.0)
lower left  (53.0, 50.0)
lower right  (737.99914260609, -19614.885517133785)
upper right  (738.0000017388409, 39.78002702526123)
upper left  (-14.0, 50.0)
lower left  (-16.0, 50.0)
lower right  (738.0004713515863, 10783.266913592792)
upper right  (738.0000017388409, 39.78002702526123)
upper left  (-14.0, 50.0)
lower left  (30.0, 50.0)
lower right  (738.0004425266627, 10123.829553977943)
upper right  (738.0000343236762, 785.2341374496935)
upper left  (-759.0, 50.0)
lower left  (738.0000343236762, 785.2341374496935)
lower right  (737.9999098922048, -2061.4260041225234)
upper right  (880.0, 50.0)
upper left  (-759.0, 50.0)
lower left  (53.0, 50.0)
lower right  (737.99914260609, -1

In [13]:
if lines is not None:
    for i in range(0, len(lines)):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        a = math.cos(theta)
        b = math.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
        pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
        cv.line(cdst, pt1, pt2, (0,0,255), 3, cv.LINE_AA)

In [14]:
pairs = []
degs = np.linspace(0, np.pi, 91)
l = [[] for _ in range(len(degs)-1)]
for i in range(len(lines)):

    deg = lines[i][0][1]
    rho = lines[i][0][0]
    id = np.where(degs>deg)[0][0]-1
    if len(np.where(abs(l[id] - rho) < 20)[0]) == 0:
        l[id].append(lines[i][0])
    
    for j in range(i+1, len(lines)):
        # if line one deg minus line two deg is in one pi/180th of pi/2
        if abs(lines[i][0][1]- lines[j][0][1]) < math.pi/2 + math.pi/360 and abs(lines[i][0][1]- lines[j][0][1]) > math.pi/2 - math.pi/360:
            pairs.append([lines[i][0], lines[j][0]])


num = []
val = []


for i in range(len(l)):
    # if len(l[i]) != 0:
        # print(len(l[i]))
        # print(degs[i])
    if len(l[i]) > 0:
        if len(val) != 0:
            if degs[i] > np.pi/2:
                if abs(np.pi - degs[i]) in val:
                    val[val.index(abs(np.pi - degs[i]))] += len(l[i])
                    continue
            if degs[i-1] == val[-1]:
                num[-1] += len(l[i])
                continue
        num.append(len(l[i]))
        val.append(degs[i])


max_deg = np.argmax(num)
s_max = abs(max_deg-1)
for i in range(len(num)):
    if i == max_deg:
        continue
    if num[i] > num[s_max]:
        s_max = i
deg_shift = (max(val[max_deg],  val[s_max]) -  min(val[max_deg],  val[s_max]) ) % (np.pi/2)
rotation = min(val[max_deg],  val[s_max]) 

print(val)
print(num)
print(f"Degree shift of: {deg_shift}")
print(f"Rotation of: {rotation*180/np.pi}")


[0.0, 0.06981317007977318, 1.4660765716752369, 4.53588974175501, 1.6406094968746698, 1.710422666954443, 2.9670597283903604, 3.036872898470133, 3.1066860685499065]
[11, 11, 4, 10, 5, 1, 6, 6, 5]
Degree shift of: 0.06981317007977318
Rotation of: 0.0


In [15]:

num = []
val = []

for i in range(len(l)):
    # if len(l[i]) != 0:
        # print(len(l[i]))
        # print(degs[i])
    if len(l[i]) > 0:
        if len(val) != 0:
            if degs[i] > np.pi/2:
                if abs(np.pi - degs[i]) in val:
                    val[val.index(abs(np.pi - degs[i]))] += len(l[i])
                    continue
            if degs[i-1] == val[-1]:
                num[-1] += len(l[i])
                continue
        num.append(len(l[i]))
        val.append(degs[i])


max_deg = np.argmax(num)
s_max = abs(max_deg-1)
for i in range(len(num)):
    if i == max_deg:
        continue
    if num[i] > num[s_max]:
        s_max = i
deg_shift = (max(val[max_deg],  val[s_max]) -  min(val[max_deg],  val[s_max]) ) % (np.pi/2)
rotation = min(val[max_deg],  val[s_max]) 

print(val)
print(num)
print(f"Degree shift of: {deg_shift}")
print(f"Rotation of: {rotation*180/np.pi}")

[0.0, 0.06981317007977318, 1.4660765716752369, 4.53588974175501, 1.6406094968746698, 1.710422666954443, 2.9670597283903604, 3.036872898470133, 3.1066860685499065]
[11, 11, 4, 10, 5, 1, 6, 6, 5]
Degree shift of: 0.06981317007977318
Rotation of: 0.0


In [16]:
img = cv.cvtColor(dst, cv.COLOR_GRAY2BGR)

for i in range(len(l)):
    print(l[i])
    for k in range(len(l[i])):
        print(l[i][k])
        print(degs[i])
        rho = l[i][k][0]
        theta = l[i][k][1]
        a = math.cos(theta)
        b = math.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
        pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
        
        
        cv.line(img, pt1, pt2, (255/len(l)*(i+1),0,255/(i+1)), 3, cv.LINE_AA)
    if len(l[i]) != 0:
        cv.imshow("img", img)
        cv.waitKey()
cv.destroyAllWindows()

[array([50.,  0.], dtype=float32), array([780.,   0.], dtype=float32), array([8.0000000e+02, 3.4906585e-02], dtype=float32), array([689.,   0.], dtype=float32), array([2.1000000e+01, 1.7453292e-02], dtype=float32), array([115.,   0.], dtype=float32), array([7.0000000e+01, 3.4906585e-02], dtype=float32)]
[50.  0.]
0.0
[780.   0.]
0.0
[8.0000000e+02 3.4906585e-02]
0.0
[689.   0.]
0.0
[2.1000000e+01 1.7453292e-02]
0.0
[115.   0.]
0.0
[7.0000000e+01 3.4906585e-02]
0.0


[array([5.500000e+01, 5.235988e-02], dtype=float32), array([8.070000e+02, 5.235988e-02], dtype=float32), array([7.500000e+01, 6.981317e-02], dtype=float32), array([8.270000e+02, 6.981317e-02], dtype=float32)]
[5.500000e+01 5.235988e-02]
0.03490658503988659
[8.070000e+02 5.235988e-02]
0.03490658503988659
[7.500000e+01 6.981317e-02]
0.03490658503988659
[8.270000e+02 6.981317e-02]
0.03490658503988659
[array([77.        ,  0.08726646], dtype=float32), array([50.        ,  0.08726646], dtype=float32), array([8.300000e+02, 8.726646e-02], dtype=float32), array([9.800000e+01, 8.726646e-02], dtype=float32)]
[77.          0.08726646]
0.06981317007977318
[50.          0.08726646]
0.06981317007977318
[8.300000e+02 8.726646e-02]
0.06981317007977318
[9.800000e+01 8.726646e-02]
0.06981317007977318
[array([99.        ,  0.10471976], dtype=float32), array([8.5000000e+02, 1.0471976e-01], dtype=float32), array([79.        ,  0.12217305], dtype=float32), array([51.        ,  0.10471976], dtype=float32), a

In [17]:
def intersection(L1, L2):
    D  = -L1[0]  + L2[0]
    Dx = L1[1]  - L2[1]
    Dy = -L1[0] * L2[1] + L1[1] * L2[0]
    if D != 0:
        x = Dx / D
        y = Dy / D
        return y, x
    else:
        return False
    

def intersectionFromPair(pair):
    if math.sin(pair[0][1]) == 0:
        if math.sin(pair[1][1] == 0):
            return False
        x = -pair[0][0] if pair[0][1] > 1 else pair[0][0] 
        y = pair[1][0]
        return y, x
    if math.sin(pair[1][1]) == 0:
        x = -pair[1][0] if pair[1][1] > 1 else pair[1][0] 
        y = pair[0][0]
        return y, x

    l_1 = (-math.cos(pair[0][1])/math.sin(pair[0][1]), pair[0][0]/math.sin(pair[0][1])) #y = l_1[0]*x+l_1[1]
    l_2 = (-math.cos(pair[1][1])/math.sin(pair[1][1]), pair[1][0]/math.sin(pair[1][1])) #y = l_1[0]*x+l_1[1]
    return intersection(l_1, l_2)

def createLine(rho, theta, img, color = (0, 0, 255)):
    a = math.cos(theta)
    b = math.sin(theta)
    x0 = a * rho
    y0 = b * rho
    pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
    pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
    cv.line(img, pt1, pt2, color, 3, cv.LINE_AA)

class rectangle:

    def __init__(self, p_1, p_2) -> None:

        self.p_1 = (int(min(p_1[0], p_2[0])), int(min(p_1[1], p_2[1])))
        self.p_2 = (int(max(p_1[0], p_2[0])), int(max(p_1[1], p_2[1])))
        if p_1[0] < p_2[0] and p_1[1]< p_2[1]:

            self.p_1 = p_1
            self.p_2 = p_2

        elif p_1[0] < p_2[0]:
            self.p_1 = [p_1[0], p_2[1]]
            self.p_2 = [p_2[0], p_1[1]]
        elif p_1[1] < p_2[0]:
            self.p_1 = [p_2[0], p_1[1]]
            self.p_2 = [p_1[0], p_2[1]]
        else:
            self.p_1 = p_2
            self.p_2 = p_1


        self.p_1 = list(map(int, self.p_1))
        self.p_2 = list(map(int, self.p_2))

    def __str__(self) -> str:
        return f"{self.p_1}, {self.p_2}"

    def area(self):
        return (self.p_2[0] - self.p_1[0])*(self.p_2[1] - self.p_1[1])


    def width(self):
        return self.p_2[1] - self.p_1[1]

    def height(self):
        return self.p_2[0] - self.p_1[0]
    
    def centrum(self):
        return (int(self.p_2[0] - self.height()/2), int(self.p_2[1] - self.width()/2))


In [18]:


i = 0
max_area_rec = rectangle((0, 0), (0, 0))
while i < len(rectangles):
    print(rectangles[i].width(), rectangles[i].height(), rectangles[i].width()/rectangles[i].height() )
    # if rectangles[i].width()/rectangles[i].height() < 0.5:
    #     rectangles.pop(i)
    #     continue
    img = cv.cvtColor(dst, cv.COLOR_GRAY2BGR)
    s = (int(rectangles[i].p_1[1]), int(rectangles[i].p_1[0]))
    e = (int(rectangles[i].p_2[1]), int(rectangles[i].p_2[0]))
    img = cv.rectangle(img, s, e, (255, 0, 0), 1)
    cv.imshow("area", img)
    cv.waitKey(0)
    print(rectangles[i].area())
    if rectangles[i].area() > max_area_rec.area():
        max_area_rec = rectangles[i]
    i += 1

# # for rec in rectangles:
center_of_rectangle = max_area_rec.centrum()
print()
print(center_of_rectangle)
print()
    
img = cv.cvtColor(dst, cv.COLOR_GRAY2BGR)
img = cv.rectangle(img, (max_area_rec.p_1[0], max_area_rec.p_1[1]), (max_area_rec.p_2[0], max_area_rec.p_2[1]), (0, 0, 255), 1)
cv.imshow("rec", img)
cv.waitKey(0)


for rec in rectangles:
    if rec == max_area_rec:
        continue
    img_c = img.copy()
    c_c = rec.centrum()
    c_score_y = abs(center_of_rectangle[0] - c_c[0])/max_area_rec.height()
    c_score_x = abs(center_of_rectangle[1] - c_c[1])/max_area_rec.width()

    # print(c_score_x, c_score_y)

    s = (int(rec.p_1[0]), int(rec.p_1[1]))
    e = (int(rec.p_2[0]), int(rec.p_2[1]))
    img = cv.rectangle(img, s, e, (255, 0, 0), 1)
    cv.imshow("rec", img)
    cv.waitKey(0)



for rec in rectangles:
    img = cv.cvtColor(dst, cv.COLOR_GRAY2BGR)

    s = (int(rec.p_1[0]), int(rec.p_1[1]))
    e = (int(rec.p_2[0]), int(rec.p_2[1]))
    img = cv.rectangle(img, s, e, (0, 0, 255), 1)
    cv.imshow("img", img)
    cv.waitKey(0)


print(len(pairs))

# for i, pair in enumerate(pairs):
#     print(pair[0], pair[1])
#     if pair[0][1] >= 0 and pair[0][1] < math.pi/2:
#         rho_f = pair[0][0]
#         theta_f = pair[0][1]
#     else:
#         rho_f = pair[1][0]
#         theta_f = pair[1][1]

#     intersection1 = intersectionFromPair(pair)
#     #TODO: check if the intersection between the lines shares any coordinates:


    # for j in range(i+1, len(pairs)):
    #     img = cv.cvtColor(dst, cv.COLOR_GRAY2BGR)
    #     if pairs[j][0][1] >= 0 and pairs[j][0][1] < math.pi/2:
    #         rho_c = pairs[j][0][0]
    #         theta_c = pairs[j][0][1]
    #     else:
    #         rho_c = pairs[j][1][0]
    #         theta_c = pairs[j][1][1]

    #     print(intersection1)
    #     intersection2 = intersectionFromPair(pairs[j])
    #     print(intersection2)
    #     createLine(pair[0][0], pair[0][1], img, (0, 0, 255))
    #     createLine(pair[1][0], pair[1][1], img, (0, 0, 255))
    #     # print(pairs[j][1][0])
    #     createLine(pairs[j][0][0], pairs[j][0][1], img, (255, 0, 0))
    #     createLine(pairs[j][1][0], pairs[j][1][1], img, (255, 0, 0))
      
    #     # l_1 = (-math.cos(pair[0][1])/math.sin(pair[0][1]), pair[0][0]/math.sin(pair[0][1])) #y = l_1[0]*x+l_1[1]
    #     # l_2 = (-math.cos(pair[1][1])/math.sin(pair[1][1]), pair[1][0]/math.sin(pair[1][1])) #y = l_1[0]*x+l_1[1]
    #     # i = intersection(l_1, l_2)
    #     # if abs(theta - pairs[j][0][1]) < math.pi/180:
    #     #     if abs(180 + (theta - 90) - pairs[j][1][1]) > math.pi/180:
    #     #         createLine(pair[0][0], pair[0][1], img, (0, 0, 255))
    #     #         createLine(pair[1][0], pair[1][1], img, (0, 0, 255))
    #     #         print(pairs[j][1][0])
    #     #         createLine(pairs[j][0][0], pairs[j][0][1], img, (255, 0, 0))
    #     #         createLine(pairs[j][1][0], pairs[j][1][1], img, (255, 0, 0))
    #     #         it += 1
    #     cv.imshow("img", img)
    #     cv.waitKey(0)
    #     #         break

    #     # elif abs(theta - pairs[j][1][1]) < math.pi/180:
    #     #     if abs(180 + (theta - 90) - pairs[j][0][1]) > math.pi/180:
    #     #         createLine(pair[0][0], pair[0][1], img, (0, 0, 255))
    #     #         createLine(pair[1][0], pair[1][1], img, (0, 0, 255))
    #     #         createLine(pairs[j][0][0], pairs[j][0][1], img, (255, 0, 0))
    #     #         createLine(pairs[j][1][0], pairs[j][1][1], img, (255, 0, 0))
    #     #         it+=1
    #     #         cv.imshow("img", img)
    #     #         cv.waitKey(0)
    #     #         break        
    #     if it > 10:
    #         break

    # if it > 10:
    #     break

cv.destroyAllWindows()



    # Find the theta that is 0 < theta <= pi/2
    # Then fint the theta_2 that is theta - pi/2 

    

    # if pair[0][1] < pair[1][1]:
    #     orientation =
    #  pair[1][1]
    #     h_pair = pair[1][1]
    #     l_pair = pair[0][1]
    # else:
    #     orientation = pair[0][1]
    #     h_pair = pair[0][1]
    #     l_pair = pair[1][1]
    



(0, 0)

2462
